<a href="https://colab.research.google.com/github/garvit26/finger_classification/blob/main/finger_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --user tensorflow
!pip install --user keras
!pip install opencv-python

In [ ]:
import tensorflow as tf
import os
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.initializers import he_normal

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path='/content/drive/My Drive/New folder/train'
data = tf.keras.utils.image_dataset_from_directory(path)
image_size = (256, 256)
batch_size = 5

num_classes = 5
train_datagen = ImageDataGenerator(  rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(
        path,
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical',
        )

Found 125 files belonging to 2 classes.
Found 125 images belonging to 2 classes.


In [ ]:
# Build the CNN model
model = Sequential()

model.add(Conv2D(32, (3, 3), 1, activation='relu', input_shape=(256, 256, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D())

model.add(Conv2D(64, (3, 3), 1, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D())

model.add(Conv2D(128, (3, 3), 1, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D())

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))  # Add dropout to reduce overfitting
model.add(Dense(1, activation='sigmoid'))
model.compile('adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(
    train_generator,
    epochs=5
)

Epoch 1/5
25/25 [==============================] - 45s 2s/step - loss: 1.8708 - accuracy: 0.5000
Epoch 2/5
25/25 [==============================] - 46s 2s/step - loss: 3.3370 - accuracy: 0.5000
Epoch 3/5
25/25 [==============================] - 45s 2s/step - loss: 6.9736 - accuracy: 0.5000
Epoch 4/5
25/25 [==============================] - 44s 2s/step - loss: 8.0165 - accuracy: 0.5000
Epoch 5/5
25/25 [==============================] - 46s 2s/step - loss: 8.3626 - accuracy: 0.5000


mobilenet

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
img_size = (224, 224)
train_data = tf.keras.utils.image_dataset_from_directory(
    path,
    batch_size=batch_size,
    image_size=img_size,
    validation_split=0.2,
    subset="training",
    seed=42
)
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(
        path,
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical')

Found 125 files belonging to 2 classes.
Using 100 files for training.
Found 125 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Load MobileNet model without the top (classification) layer
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add your custom top layers for classification
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(2, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_generator, epochs=10)

Epoch 1/10
25/25 [==============================] - 47s 1s/step - loss: 0.3574 - accuracy: 0.9120
Epoch 2/10
25/25 [==============================] - 34s 1s/step - loss: 0.5450 - accuracy: 0.9040
Epoch 3/10
25/25 [==============================] - 35s 1s/step - loss: 0.3003 - accuracy: 0.9600
Epoch 4/10
25/25 [==============================] - 35s 1s/step - loss: 0.0162 - accuracy: 1.0000
Epoch 5/10
25/25 [==============================] - 34s 1s/step - loss: 0.1604 - accuracy: 0.9600
Epoch 6/10
25/25 [==============================] - 35s 1s/step - loss: 0.0828 - accuracy: 0.9920
Epoch 7/10
25/25 [==============================] - 37s 1s/step - loss: 0.0779 - accuracy: 0.9840
Epoch 8/10
25/25 [==============================] - 34s 1s/step - loss: 0.0643 - accuracy: 0.9760
Epoch 9/10
25/25 [==============================] - 34s 1s/step - loss: 0.0258 - accuracy: 0.9920
Epoch 10/10
25/25 [==============================] - 34s 1s/step - loss: 0.0578 - accuracy: 0.9840


In [ ]:
test_path='/content/drive/My Drive/New folder/test'
# Data loading for testing
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical')

Found 27 images belonging to 2 classes.


In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

6/6 [==============================] - 8s 2s/step - loss: 0.0638 - accuracy: 0.9259
Test Accuracy: 92.59%
